In [ ]:
# Loading Libraries, Remember to run this cell!
library(tidyverse)
library(repr)
library(tidymodels)

# Project Proposal

In [3]:
# Loading in the data
player_stats <- read_csv("player_stats.csv")
player_stats

ERROR: Error in read_csv("player_stats.csv"): could not find function "read_csv"
